In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import ElasticNet, Lasso,  BayesianRidge, LassoLarsIC
from sklearn.ensemble import RandomForestRegressor,  GradientBoostingRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
import xgboost as xgb
import itertools

In [58]:
data = pd.read_csv("data.csv")
data.head()

,year,month,day,anger_score,disgust_score,fear_score,joy_score,sadness_score,anger_count,disgust_count,fear_count,joy_count,sadness_count,close,open,high,low,change
0,2011,4,11,23.678598,1.862068,3.365532,129.260966,22.046661,28,2,4,150,26,12381.11,12380.43,12476.93,12319.35,0.0001
1,2011,4,12,23.436641,1.568504,1.807040,110.107599,28.552481,28,2,2,128,33,12263.58,12381.04,12381.04,12185.09,-0.9500
2,2011,4,13,17.595715,3.441165,3.351268,103.535217,17.206766,21,4,4,121,21,12270.99,12263.73,12372.94,12203.63,0.0006
3,2011,4,14,16.065135,4.921378,5.183304,102.417106,15.457086,19,6,6,121,18,12285.15,12270.24,12332.71,12137.98,0.0012
4,2011,4,15,16.869910,1.555661,3.334624,121.630811,14.709462,20,2,4,141,18,12341.83,12285.45,12402.61,12238.34,0.0046


In [ ]:
#일단 score랑 count만가지고 train해보자

In [28]:
train_data = data.iloc[:,3:13]
train_label = data.iloc[:,13]
train_label.head()

0    12381.11
1    12263.58
2    12270.99
3    12285.15
4    12341.83
Name: close, dtype: float64

In [29]:
train_data = pd.concat([train_data,data['open']],axis=1)

In [30]:
train_data.head()

,anger_score,disgust_score,fear_score,joy_score,sadness_score,anger_count,disgust_count,fear_count,joy_count,sadness_count,open
0,23.678598,1.862068,3.365532,129.260966,22.046661,28,2,4,150,26,12380.43
1,23.436641,1.568504,1.807040,110.107599,28.552481,28,2,2,128,33,12381.04
2,17.595715,3.441165,3.351268,103.535217,17.206766,21,4,4,121,21,12263.73
3,16.065135,4.921378,5.183304,102.417106,15.457086,19,6,6,121,18,12270.24
4,16.869910,1.555661,3.334624,121.630811,14.709462,20,2,4,141,18,12285.45


In [31]:
#3일 뒤를 예측
train_data = train_data.iloc[:-3,:]
train_label = train_label.iloc[3:]
print(train_data.shape)
print(train_label.shape)

(369, 11)
(369,)


(369,)

In [32]:
scaler = RobustScaler()
scale_data = scaler.fit_transform(train_data)

In [33]:
logged_train_label = np.log1p(train_label)
logged_train_label = np.array(logged_train_label)

In [34]:
def rmsle_cv(model, n_folds=5):
    kf = KFold(n_folds, shuffle=True).get_n_splits(scale_data)
    rmse= np.sqrt(-cross_val_score(model, scale_data,logged_train_label, scoring="neg_mean_squared_error", cv = kf))
    return(rmse.mean())


In [49]:
model_xgb = xgb.XGBRegressor(colsample_bytree=0.4603, gamma=0.09,
                             learning_rate=0.009, 
                              n_estimators=100,
                             reg_alpha=0.4640, reg_lambda=0.8571,
                             subsample=0.5213,
                             random_state =7, nthread = -1)
xgb_m = xgb.XGBRegressor(leraning_rate = 0.05, n_estimators= 1000)
GBoost = GradientBoostingRegressor(max_depth=2, learning_rate=0.005,min_samples_leaf=15, min_samples_split=10, 
                                   loss='huber', random_state =5)

KRR = KernelRidge(alpha=0.1, kernel='polynomial', degree=1, coef0=5)

ENet = ElasticNet(alpha=0.005, l1_ratio=.6, random_state=3)

lasso = Lasso(alpha =0.005, random_state=1)

RForest = RandomForestRegressor(n_estimators=100,max_depth=2,min_samples_leaf=2\
                               ,min_weight_fraction_leaf=0.5,max_leaf_nodes=2\
                                ,random_state=6)

In [50]:
print(rmsle_cv(model_xgb))
print(rmsle_cv(GBoost))
print(rmsle_cv(KRR))
print(rmsle_cv(ENet))
print(rmsle_cv(lasso))

3.6341967179036887
0.0422841419325357
0.021313119649035477
0.023543197815842378
0.025565595199804864


In [45]:
class AveragingModels(BaseEstimator, RegressorMixin, TransformerMixin):
    def __init__(self, models):
        self.models = models
        
    # we define clones of the original models to fit the data in
    def fit(self, X, y):
        self.models_ = [clone(x) for x in self.models]
        
        # Train cloned base models
        for model in self.models_:
            model.fit(X, y)

        return self
    
    #Now we do the predictions for cloned models and average them
    def predict(self, X):
        predictions = np.column_stack([
            model.predict(X) for model in self.models_
        ])
        return np.mean(predictions, axis=1)

In [288]:
averaged_models = AveragingModels(models = (ENet, KRR,GBoost,RForest,lasso ,model_xgb))

rmsle_cv(averaged_models)

0.055375069546392666

In [ ]:
#5개 모두시 떨어짐
#model_xgb, ENet, KRR, lasso . 0.056998897272156866
#model_xgb, ENet, KRR          0.0569513660909768 
#model_xgb, ENet , lasso       0.056966904668101906
#model_xgb, ENet,              0.05693818530913114

In [250]:
#scale_data
#logged_train_label


In [37]:

class StackingAveragedModels(BaseEstimator, RegressorMixin, TransformerMixin):
    def __init__(self, base_models, meta_model, n_folds=5):
        self.base_models = base_models
        self.meta_model = meta_model
        self.n_folds = n_folds
    # We again fit the data on clones of the original models
    def fit(self, X, y):
        self.base_models_ = [list() for x in self.base_models]
        self.meta_model_ = clone(self.meta_model)
        kfold = KFold(n_splits=self.n_folds, shuffle=True)
        
        # Train cloned base models then create out-of-fold predictions
        # that are needed to train the cloned meta-model
        out_of_fold_predictions = np.zeros((X.shape[0], len(self.base_models)))
        
        for i, model in enumerate(self.base_models):
            
            for train_index, holdout_index in kfold.split(X, y):
                instance = clone(model)
                
                self.base_models_[i].append(instance)
                
                instance.fit(X[train_index], y[train_index])
                y_pred = instance.predict(X[holdout_index])
                out_of_fold_predictions[holdout_index, i] = y_pred
      
        # Now train the cloned  meta-model using the out-of-fold predictions as new feature
        self.meta_model_.fit(out_of_fold_predictions, y)
        
        return self
   
    #Do the predictions of all base models on the test data and use the averaged predictions as 
    #meta-features for the final prediction which is done by the meta-model
    def predict(self, X):
        meta_features = np.column_stack([
            np.column_stack([model.predict(X) for model in base_models]).mean(axis=1)
            for base_models in self.base_models_ ])
        return self.meta_model_.predict(meta_features)


In [28]:
stacked_averaged_models = StackingAveragedModels(base_models = (ENet,KRR,lasso,RForest),meta_model =  GBoost)
rmsle_cv(stacked_averaged_models)

0.054000115716498764

In [86]:
#base_models = (ENet,lasso,KRR,model_xgb),meta_model =  GBoost 0.053922348347839634
#base_models = (ENet,lasso,KRR,model_xgb),meta_model =  GBoost 0.054324193590506976
#base_models = (ENet,KRR),meta_model =  GBoost 0.0537208976739747

(369, 10)

In [297]:
#scale_data
#logged_train_label
pd.DataFrame(scale_data).to_csv("Last_train.csv",index=False)
pd.DataFrame(logged_train_label).to_csv("Last_label.csv",index=False)

In [11]:
train = scale_data[:-100]
test = scale_data[-100:]
train_label = train_label[:-100]
test_label = train_label[-100:]

In [30]:
stacked_averaged_models = StackingAveragedModels(base_models = (ENet,KRR,lasso,RForest),meta_model =  GBoost)
stacked_averaged_models.fit(train,logged_train_label[:-100]) #train_label은 log를 넣었었다가 안되서 그냥값을 넣어본거임.

StackingAveragedModels(base_models=(ElasticNet(alpha=0.002, copy_X=True, fit_intercept=True, l1_ratio=0.6,
      max_iter=1000, normalize=False, positive=False, precompute=False,
      random_state=3, selection='cyclic', tol=0.0001, warm_start=False), KernelRidge(alpha=0.1, coef0=5, degree=1, gamma=None, kernel='polynomia..._estimators=100, n_jobs=1,
           oob_score=False, random_state=6, verbose=0, warm_start=False)),
            meta_model=GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.005, loss='huber', max_depth=2,
             max_features=None, max_leaf_nodes=None,
             min_impurity_decrease=0.0, min_impurity_split=None,
             min_samples_leaf=15, min_samples_split=10,
             min_weight_fraction_leaf=0.0, n_estimators=100,
             presort='auto', random_state=5, subsample=1.0, verbose=0,
             warm_start=False),
            n_folds=5)

In [22]:
xgb_m = xgb.XGBRegressor(leraning_rate = 0.01, n_estimators= 100)
rmsle_cv(xgb_m)

0.059743052171724174

In [51]:

def train_and_predict(model,X_data,Y_data,cut_num=5):
    '''
    split data into 5 groups
    
    1.fit group 1
    
    2.predict group i and fit group i ( i = 2,3,4,5)
    
    3.and result is (model, predict_result)
    '''
    
    ind_lst = [int((len(X_data)/cut_num) * i) for i in range(6)]
    model.fit(X_data[ind_lst[0]:ind_lst[4]],Y_data[ind_lst[0]:ind_lst[4]])
    result = [Y_data[:ind_lst[4]]]

    for i in range(ind_lst[4],ind_lst[5]):
        temp = model.predict(X_data[i].reshape(1,-1))
        result.append(temp)
        #model.fit(X_data[i].reshape(1,-1),Y_data[i].reshape(1,-1))
    
    result = np.array(list(itertools.chain(*result)))
    return result
result = train_and_predict(lasso,scale_data,logged_train_label)

In [ ]:
def train_and

In [52]:
result = np.expm1(result)

In [53]:
#res = stacked_averaged_models.predict(test)
#res = np.expm1(res)
real_value = np.expm1(logged_train_label)

In [54]:
x_range = [x for x in range(1,len(real_value)+1)]
x2_range = [x for x in range(1, len(result)+1)]

In [163]:
real_value.shape

(369,)

In [55]:
import matplotlib.pyplot as plt

In [57]:
%matplotlib tk
plt.figure(1)
plt.plot(x_range,real_value)
plt.plot(x2_range,result)